In [3]:
import os
import openai

In [4]:
import tiktoken

In [5]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

open.api_key = os.environ['OPENAI_API_KEY']

In [6]:
def complete_prompt(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.5,
        max_tokens=60,
        stop=["."],
    )
    return response.choices[0].message.content

with open("dataset.txt") as phrases_file, open("no_cue_prompts.txt") as prompts_file:
    phrases = phrases_file.readlines()
    prompts = prompts_file.readlines()
    complete_sentences = []
        
    for prompt in prompts: 
        generated_text = complete_prompt(prompt)
        complete_sentences.append(prompt + generated_text)
        print(f"No Cue Prompt:{prompt}")
        print(f"Generated Text:{generated_text}\n")

No Cue Prompt:The employees knew the contract 

Generated Text:inside and out, as they had spent hours reviewing and discussing it during their training sessions

No Cue Prompt:The mechanic accepted the car 

Generated Text:for repair and began diagnosing the issue

No Cue Prompt:The old man recalled the nurse

Generated Text: who had cared for him during his stay in the hospital

No Cue Prompt:The traveller heard the clock 

Generated Text:ticking in the distance, a constant reminder of the passing time

No Cue Prompt:The journalist confirmed the story 

Generated Text:The journalist verified the accuracy of the story

No Cue Prompt:The worker maintained the walls

Generated Text: by regularly inspecting for cracks or damage and promptly repairing any issues

No Cue Prompt:The apprentice forgot the bicycle 

Generated Text:he was supposed to deliver to the customer

No Cue Prompt:The committee mentioned the issue

Generated Text: during their meeting

No Cue Prompt:The army found the 

In [133]:
import nltk
from nltk import FreqDist
from nltk.tokenize import word_tokenize
import re

In [156]:
with open ("np.txt") as np_sentences, open ("s.txt") as s_sentences:
    np_sentences = np_sentences.read().lower()
    s_sentences = s_sentences.read().lower()
    
    np_tokens = nltk.word_tokenize(np_sentences)
    fdist_np = FreqDist(np_tokens)
    re_pattern = r'\w+'
    np_clean = []
    
    s_tokens = nltk.word_tokenize(s_sentences)
    fdist_s = FreqDist(s_tokens)
    re_pattern = r'\w+'
    s_clean = []
    
    for word in np_tokens:
        if re.match(re_pattern, word):
            np_clean.append(word)
            
    for word in s_tokens:
        if re.match(re_pattern, word):
            s_clean.append(word)

In [157]:
print(np_clean)

['the', 'old', 'man', 'recalled', 'the', 'nurse', 'who', 'had', 'cared', 'for', 'him', 'during', 'his', 'recovery', 'at', 'the', 'hospital', 'the', 'worker', 'maintained', 'the', 'walls', 'by', 'regularly', 'inspecting', 'for', 'cracks', 'or', 'damage', 'patching', 'up', 'any', 'areas', 'that', 'needed', 'repair', 'and', 'repainting', 'or', 'sealing', 'as', 'necessary', 'the', 'apprentice', 'forgot', 'the', 'bicycle', 'at', 'the', 'shop', 'and', 'had', 'to', 'walk', 'home', 'instead', 'the', 'army', 'found', 'the', 'supplies', 'they', 'had', 'been', 'searching', 'for', 'in', 'a', 'hidden', 'bunker', 'deep', 'in', 'the', 'forest', 'the', 'umpire', 'warned', 'the', 'spectators', 'to', 'refrain', 'from', 'using', 'any', 'inappropriate', 'language', 'or', 'behavior', 'during', 'the', 'game', 'the', 'tourists', 'saw', 'the', 'palace', 'and', 'were', 'amazed', 'by', 'its', 'grandeur', 'and', 'beauty', 'the', 'scientists', 'proved', 'the', 'theory', 'through', 'extensive', 'research', 'and', 

In [158]:
print(len(np_clean))

222


In [159]:
print(s_clean)

['the', 'employees', 'knew', 'the', 'contract', 'inside', 'and', 'out', 'as', 'they', 'had', 'spent', 'weeks', 'studying', 'and', 'reviewing', 'its', 'contents', 'the', 'mechanic', 'accepted', 'the', 'car', 'into', 'the', 'shop', 'for', 'repairs', 'the', 'traveller', 'heard', 'the', 'clock', 'ticking', 'in', 'the', 'distance', 'its', 'steady', 'rhythm', 'echoing', 'through', 'the', 'empty', 'streets', 'the', 'committee', 'mentioned', 'the', 'issue', 'the', 'committee', 'mentioned', 'the', 'issue', 'of', 'declining', 'attendance', 'at', 'community', 'events', 'and', 'discussed', 'potential', 'strategies', 'to', 'increase', 'participation', 'the', 'soldiers', 'remembered', 'the', 'town', 'they', 'had', 'liberated', 'just', 'a', 'few', 'days', 'before']


In [160]:
print(len(s_clean))

81


In [165]:
fdist_np = FreqDist(np_clean)
fdist_np.most_common(10)

[('the', 40),
 ('and', 7),
 ('for', 6),
 ('to', 5),
 ('had', 4),
 ('or', 4),
 ('during', 3),
 ('at', 3),
 ('by', 3),
 ('any', 3)]

In [166]:
fdist_s = FreqDist(s_clean)
fdist_s.most_common(10)

[('the', 15),
 ('and', 3),
 ('they', 2),
 ('had', 2),
 ('its', 2),
 ('committee', 2),
 ('mentioned', 2),
 ('issue', 2),
 ('employees', 1),
 ('knew', 1)]

In [7]:
complete_sentences

['The employees knew the contract \ninside and out, as they had spent hours reviewing and discussing it during their training sessions',
 'The mechanic accepted the car \nfor repair and began diagnosing the issue',
 'The old man recalled the nurse\n who had cared for him during his stay in the hospital',
 'The traveller heard the clock \nticking in the distance, a constant reminder of the passing time',
 'The journalist confirmed the story \nThe journalist verified the accuracy of the story',
 'The worker maintained the walls\n by regularly inspecting for cracks or damage and promptly repairing any issues',
 'The apprentice forgot the bicycle \nhe was supposed to deliver to the customer',
 'The committee mentioned the issue\n during their meeting',
 'The army found the supplies\n that had been hidden in a remote cave deep in the forest',
 'The umpire warned the spectators\n to refrain from using offensive language and to respect the players on the field',
 'The coach discovered the pla

In [8]:
cleaned_sentences = [sentence.replace('\n', '') for sentence in complete_sentences]
cleaned_sentences

['The employees knew the contract inside and out, as they had spent hours reviewing and discussing it during their training sessions',
 'The mechanic accepted the car for repair and began diagnosing the issue',
 'The old man recalled the nurse who had cared for him during his stay in the hospital',
 'The traveller heard the clock ticking in the distance, a constant reminder of the passing time',
 'The journalist confirmed the story The journalist verified the accuracy of the story',
 'The worker maintained the walls by regularly inspecting for cracks or damage and promptly repairing any issues',
 'The apprentice forgot the bicycle he was supposed to deliver to the customer',
 'The committee mentioned the issue during their meeting',
 'The army found the supplies that had been hidden in a remote cave deep in the forest',
 'The umpire warned the spectators to refrain from using offensive language and to respect the players on the field',
 'The coach discovered the player had exceptional 

In [78]:
def categorize_sentences(messages,
                model="gpt-3.5-turbo",
                temperature=0,
                max_tokens=750):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content

In [79]:
sentences = ',\n'.join([
    f'{{"sentence": "{sentence}", "is_NP": true/false, "is_S": true/false}}'
    for sentence in cleaned_sentences
])

system_message = """
You will be provided with a list of sentences.
For each sentence, classify whether it is a noun phrase (NP) or a sentential complement (S).
NP acts as the direct object of the main verb, while S acts as the embedded subject in an upcoming subordinate clause.
Provide the output in JSON format with the following structure:

{
    "sentences": [
        """ + sentences + """
    ]
}
"""

messages = [
    {'role': 'system', 'content': system_message}
]

response = categorize_sentences(messages)
with open("output.json", "w") as file:
    file.write(response)

In [81]:
print(response)

{
    "sentences": [
        {"sentence": "The employees knew the contract inside and out, as they had spent hours reviewing and discussing it during their training sessions", "is_NP": false, "is_S": true},
        {"sentence": "The mechanic accepted the car for repair and began diagnosing the issue", "is_NP": true, "is_S": false},
        {"sentence": "The old man recalled the nurse who had cared for him during his stay in the hospital", "is_NP": true, "is_S": false},
        {"sentence": "The traveller heard the clock ticking in the distance, a constant reminder of the passing time", "is_NP": true, "is_S": false},
        {"sentence": "The journalist confirmed the story The journalist verified the accuracy of the story", "is_NP": true, "is_S": false},
        {"sentence": "The worker maintained the walls by regularly inspecting for cracks or damage and promptly repairing any issues", "is_NP": true, "is_S": false},
        {"sentence": "The apprentice forgot the bicycle he was suppose